# Distributed Protocols

A *distributed protocol* is a specification for a computation performed jointly by several communicating *parties*. In this book, we'll be concerned with *synchronous* protocols, in which communication between the parties occurs at pre-specified times.

## Choreographic Programming and PyChor

In this book, we'll implement protocols using the PyChor library. PyChor is a Python library for [*choreographic programming*](https://en.wikipedia.org/wiki/Choreographic_programming), a programming paradigm in which a single global program specifies the computations and synchronous communications of all parties.

You can install PyChor using `pip`:

```
pip install pychor
```

The PyChor library enables the implementation of protocols as Python programs in a choreographic style. PyChor adds *located values*---values known to a single party participating in the protocol---as special kinds of Python values. For example, the following choreography consists of two parties (`p1` and `p2`), and `p1` knows that the value of the variable `x` is 5 (`x` is *located* at `p1`).

In [1]:
import pychor as pc
p1 = pc.Party('p1')
p2 = pc.Party('p2')

with pc.LocalBackend():
    x = 5@p1
    print(x)

5@{p1}


Here, the `with` block starts a new choreography using the *local backend*. The local backend simulates the execution of the protocol on a single computer, without using any actual network communication. The `@` symbol is used in PyChor to indicate that a value should be located at a particular location, so `5@p1` is the value 5 located at `p1`.

To send a value to another party, we can use the `send` operator. The expression `x.send(p, q)` sends the value of `x` from party `p` to party `q`. After this operation, *both* `p` and `q` will know the value of `x`---it is *located at both parties*. For example, the following protocol constructs a value located at `p1`, then `p1` sends that value to `p2`.

In [2]:
with pc.LocalBackend():
    x = 5@p1
    x.send(p1, p2)
    print(x)

5@{p2, p1}


One example of a simple protocol is one for summing two numbers. Imagine `p1` has the number 5, and `p2` has the number 6, and the parties would like to compute the sum of these two numbers. To accomplish this, each party can send their number to the other party, and both parties can add them together.

In [3]:
with pc.LocalBackend():
    # initialize the input values
    v1 = 3@p1
    v2 = 4@p2

    # Round 1: Each party sends their input to the other party
    v1.send(p1, p2)
    v2.send(p2, p1)

    # Round 2: Both parties sum the results and print them out
    total = v1 + v2
    print('Total sum:', total)

Total sum: 7@{p2, p1}


After round 1, the values of `v1` and `v2` are known to both parties, so the expression `v1 + v2` results in *both* parties computing the total sum in parallel. The value of `total` is therefore also known to both parties. PyChor lifts common arithmetic operations (like +) to located values, so that the expression `v1 + v2` works as expected, even though `v1` and `v2` are located values rather than regular Python numbers.

The resulting pattern of communication between the parties is depicted in the following [sequence diagram](https://en.wikipedia.org/wiki/Sequence_diagram):

```{mermaid}
sequenceDiagram
p1 ->> p2 : 3
p2 ->> p1 : 4
```

This protocol requires both parties to reveal their input number to the other party, and does not provide any kind of security. We'll see how to add security (and perform more complicated computation) in later chapters.

# Homework: extend this to $n$ parties